# Prepossessing

### (Buggy) Construct dict from seqs without U,X

In [ ]:
seqs_noUX = np.array(df.sequence)
target_noUX = np.array(df.fingerprint)
print (target_noUX.shape)

# For All sequences in dataset
rows_to_delete = []
for i in range(seqs_noUX.shape[0]):
    if 'X' in seqs_noUX[i] or 'U' in seqs_noUX[i]:
        print('Removing...' + str(i))
        rows_to_delete.append(i)        
        
seqs_noUX = np.delete(seqs_noUX, rows_to_delete, 0)
target_noUX = np.delete(target_noUX, rows_to_delete)
print ("Sequence length is " + str(seqs_noUX.shape[0]))

In [1]:
# ProtVec
n_gram_size = 3
dimension = 100

# Onehot


# Whether to use simple onehot encoding for inputs
UseOnehot = True
UseReplicate = False

### Input - ProtVec Embedding

In [ ]:
import biovec

# A very simple protein sequences to fasta file conversion
def convertSeqsToFasta (seqs, filename):
    file = open(filename, 'w')
    size = seqs.shape[0]
    for i,l in zip(range(size),seqs):
        file.write('>' + str(i) + '\n')
        file.write(l + '\n') 

In [ ]:
filename = 'exclude_U_X.fasta'
convertSeqsToFasta(seqs=seqs_noUX, filename=filename)
pv = biovec.models.ProtVec(corpus_fname=filename, n=n_gram_size, size=dimension)

In [ ]:
import json

def construct_n_gram_to_vector_dict(filename, seqs, pv, n_gram_size):
    n_gram_to_vec = {}
    
    for i in range(seqs.shape[0]):
        for j in range(len(seqs[i])-(n_gram_size-1)):
            word = seqs[i][j:j+n_gram_size]
            n_gram_to_vec[word] = pv[word].tolist()
    with open(filename, 'w') as fp:
        json.dump(n_gram_to_vec, fp)  
    print ("Done")

In [ ]:
# Write
construct_n_gram_to_vector_dict(dict_path + "seqs_to_vec_noUX", seqs, pv, n_gram_size)

### Input - Onehot Encoding

In [ ]:
onehot_keys = []
for i in range(seqs.shape[0]):
    for j in range(len(seqs[i])):
        if seqs[i][j] not in onehot_keys:
            onehot_keys.append(seqs[i][j])
# import operator
# print (sorted(seqs_dict_onehot.items(), key=operator.itemgetter(1)))
onehot_keys = np.array(onehot_keys).reshape(-1)
np.random.shuffle(onehot_keys)

seqs_dict_onehot = {}
onehot_matrix = np.eye(onehot_keys.shape[0])
for i in range(onehot_keys.shape[0]):
    seqs_dict_onehot[onehot_keys[i]] = onehot_matrix[i].tolist()

In [ ]:
import json
with open(dict_path + "seqs_dict_onehot", 'w') as fp:
    json.dump(seqs_dict_onehot, fp)

### Construct feature vector

<img src="./figs/length_dist.png">

In [ ]:
if UseOnehot:
    n_gram_size = 1
    dimension = onehot_keys.shape[0]
    seqs_dict = seqs_dict_onehot

In [ ]:
import matplotlib.pyplot as plt

# Plot distribution of sequence lengths
lengths = []
for item in seqs:
    lengths.append(len(item))

fig = plt.figure(0)
ax = fig.add_subplot(111)
ax.set_title('Sequence Length Distribution')
n, bins, rectangles = ax.hist(lengths, 200)
fig.canvas.draw()
plt.show()

# Models

#### Many to Many BRNN
    1. Take average at output states
    2. activation function tanh [-1, 1]

#### Many to Many regular RNN

In [ ]:
# Build Model
model = Sequential()
# model.add()
model.add(LSTM(128, input_shape=(cutoff,dimension)))
model.add(Dropout(0.2))
model.add(Dense(cutoff, activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[])
model.summary()

In [ ]:
#Input (50,706,100) = 3,530,000 [141200000] 
#Output (50,706,1) = 35,300 [1412000]

history_of_loss = [] # Accumulated loss over epoch * num_of_samples / fold_size
histroy_of_roc = []
epoch = 5
batch_size = 50 # Minibatch size
fold_size = 1000 # Input tensor size (fold_size, cutoff, dimension)

def training_procedure():
    
    # Shuffle batch at each epoch
    X_train_shuffled, T_train_shuffled = shuffleXY(X_train, T_train)
    X_val_shuffled, T_val_shuffled = shuffleXY(X_val, T_val)
    
    # Construct validation sets
    X_Val = construct_X(seqs_dict, X_val_shuffled, cutoff, n_gram_size, dimension, len(X_val_shuffled), 0)
    T_Val = np.array(T_val_shuffled).reshape(len(T_val_shuffled), cutoff)
    
    # 1 to n-1 folds
    n = int(len(X_train) / fold_size)
    for i in range(n):
        print('[' + str(i * fold_size) + '] to [' + str((i+1) * fold_size) + ']')   
        X = construct_X(seqs_dict, X_train_shuffled, cutoff, n_gram_size, dimension, fold_size, i * fold_size)
        T = np.array(T_train_shuffled[i * fold_size: (i+1) * fold_size]).reshape(fold_size,cutoff)

#         hist = model.fit(X, T, epochs=1, batch_size=batch_size, verbose=0)
        cb = roc_callback((X,T), (X_Val, T_Val))
        hist = model.fit(X, T, epochs=1, batch_size=batch_size, verbose=0, 
                         callbacks=[cb])
        
        history_of_loss.append(hist.history['loss'])
        histroy_of_roc.extend(cb.roc_scores)


    # Last fold
    last_fold_size = len(X_train) - n * fold_size
    print('[' + str(n * fold_size) + '] to [' + str(n * fold_size + last_fold_size) + ']')   

    X = construct_X(seqs_dict, X_train_shuffled, cutoff, n_gram_size, dimension, last_fold_size, n * fold_size)
    T = np.array(T_train_shuffled[n * fold_size : n * fold_size + last_fold_size]).reshape(last_fold_size,cutoff)

    cb = roc_callback((X,T), (X_Val, T_Val))
    hist = model.fit(X, T, epochs=1, batch_size=batch_size, verbose=0, 
                         callbacks=[cb])

    # hist = model.fit(X, T, epochs=epoch, batch_size=batch_size, verbose=1, 
    #                     callbacks=[roc_callback((X,T), (X_Val, T_Val))])
    history_of_loss.append(hist.history['loss'])
    histroy_of_roc.extend(cb.roc_scores)
for i in range(epoch):
    print ('\n\n\n*** Epoch ', i+1, ' ***')
    training_procedure()

### Visualization and Backup

#### Tried out combinations of onehot/word2vec, replication/zero-padding
    1. Onehot + Replication auroc ~ [0.9] 10 epochs
<img src="./figs/bd_rp_oh_10.png">
    2. W2V + Replication auroc ~ [0.8] (!) 10 epochs
<img src="./figs/bd_rp_20.png">
    3. Onehot + Zero-padding auroc ~ [0.99] (!) 20 epochs
<img src="./figs/bd_oh_20.png">
    4. W2V + Zero-padding (Old model) auroc ~ [0.75 - 0.8] 10 epochs

<img src = "./figs/hb_lr_10.png">
<img src = "./figs/hb_lr_20.png">

In [ ]:
print ('Metrics are [ ' + str(history.history.keys()) + ' ]')

In [ ]:
print (np.array(histroy_of_roc)[:,1])